## Get Candlestick Data 

In [1]:
import requests
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
def find_candle_trend (candle_no):
    if candle_no < 10:
        return 0
    
    data = pd.read_csv(filename)
    data = data.iloc[candle_no-5:candle_no+1]
#    display (data[['Open','Close','Low','High']])
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = int(r2_score(Y, y_pred)*100)
#    print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)
    
    if coeficient > 0:
        Current_Market = 1 *  Current_Market_Fit## Bullish / Buy ##
    else:
        Current_Market = -1 * Current_Market_Fit ## Bearish / Sell ##
    
    return Current_Market

In [5]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    instrument = cfg['currency']['instrument']

In [6]:
print('Instrument:', instrument)

Instrument: EUR_USD


### The Configs for Run:

In [7]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### Select the Currency Pair

In [8]:
Load_10K_Records=True
Load_20K_Records=False

#  'EUR_USD','USD_CAD','EUR_GBP','EUR_AUD','EUR_CHF',
#  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD',
#  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','EUR_CAD',
#  'USD_CHF','CAD_CHF','NZD_CAD','AUD_USD','EUR_NZD',
#  'NZD_CHF',

currency_pairs = [instrument]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [9]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

# Get Candlesticks Data

In [10]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [11]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [12]:
if Load_10K_Records:
    
    #### Load 10K data
        
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
    datetime_object_15K = parser.parse(second_response['candles'][0]['time'])
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    
    if Load_20K_Records:

        #### Load 15K data

        date= datetime_object_15K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        third_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(third_response['candles'][0]['time'])
        third_response= third_response['candles']
        third_response.extend(second_response)

        #### Load 20K data

        date= datetime_object_20K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        fourth_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(fourth_response['candles'][0]['time'])
        fourth_response= fourth_response['candles']
        fourth_response.extend(third_response)

        response=fourth_response
    else:
        response=second_response 
else:
    response=first_response['candles']

In [13]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [14]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [15]:
data.shape

(9670, 12)

In [16]:
data.head(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-12-07,14:00:00,14,341,0,Monday,2015-12-07T14:00:00.000000000Z,9068,1.08170,1.08600,1.08054,1.08512
1,2015-12-07,18:00:00,18,341,0,Monday,2015-12-07T18:00:00.000000000Z,3733,1.08516,1.08603,1.08322,1.08374
2,2015-12-07,22:00:00,22,341,0,Monday,2015-12-07T22:00:00.000000000Z,1616,1.08366,1.08484,1.08304,1.08468
3,2015-12-08,02:00:00,2,342,1,Tuesday,2015-12-08T02:00:00.000000000Z,1568,1.08468,1.08606,1.08433,1.08606
4,2015-12-08,06:00:00,6,342,1,Tuesday,2015-12-08T06:00:00.000000000Z,6176,1.08604,1.08822,1.08532,1.08536


In [17]:
data.tail(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9665,2022-02-23,22:00:00,22,54,2,Wednesday,2022-02-23T22:00:00.000000000Z,15853,1.13080,1.13111,1.12752,1.12932
9666,2022-02-24,02:00:00,2,55,3,Thursday,2022-02-24T02:00:00.000000000Z,62258,1.12932,1.12940,1.12086,1.12380
9667,2022-02-24,06:00:00,6,55,3,Thursday,2022-02-24T06:00:00.000000000Z,78433,1.12380,1.12624,1.12090,1.12114
9668,2022-02-24,10:00:00,10,55,3,Thursday,2022-02-24T10:00:00.000000000Z,86397,1.12114,1.12124,1.11528,1.11620
9669,2022-02-24,14:00:00,14,55,3,Thursday,2022-02-24T14:00:00.000000000Z,35599,1.11619,1.11716,1.11322,1.11333


## Simple Moving Average (SMA)

In [18]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [19]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [20]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [21]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20
9665,2022-02-23,22:00:00,22,54,2,Wednesday,2022-02-23T22:00:00.000000000Z,15853,1.13080,1.13111,1.12752,1.12932,1.1323,1.1328,1.1329,-0.00298,-0.00348,-0.00358
9666,2022-02-24,02:00:00,2,55,3,Thursday,2022-02-24T02:00:00.000000000Z,62258,1.12932,1.12940,1.12086,1.12380,1.1300,1.1318,1.1325,-0.00620,-0.00800,-0.00870
9667,2022-02-24,06:00:00,6,55,3,Thursday,2022-02-24T06:00:00.000000000Z,78433,1.12380,1.12624,1.12090,1.12114,1.1272,1.1305,1.1318,-0.00606,-0.00936,-0.01066
9668,2022-02-24,10:00:00,10,55,3,Thursday,2022-02-24T10:00:00.000000000Z,86397,1.12114,1.12124,1.11528,1.11620,1.1242,1.1289,1.1308,-0.00800,-0.01270,-0.01460
9669,2022-02-24,14:00:00,14,55,3,Thursday,2022-02-24T14:00:00.000000000Z,35599,1.11619,1.11716,1.11322,1.11333,1.1208,1.1269,1.1297,-0.00747,-0.01357,-0.01637


## Price Range

In [22]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [23]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [24]:
%%time
for candle_no in data.index.tolist():
#    print (candle_no)
    data.at[candle_no, "Trend"] = find_candle_trend(candle_no)

CPU times: user 7min 19s, sys: 8.19 s, total: 7min 27s
Wall time: 7min 27s


In [25]:
data.head()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5,Trend
0,2015-12-10,18:00:00,18,344,3,Thursday,2015-12-10T18:00:00.000000000Z,3356,1.09469,1.09566,...,0.00264,0.00157,-0.00107,0,0.00060,0.00097,0.00107,0.00236,-0.00032,0.0
1,2015-12-10,22:00:00,22,344,3,Thursday,2015-12-10T22:00:00.000000000Z,1840,1.09407,1.09477,...,0.00211,0.00153,-0.00058,0,0.00083,0.00070,0.00058,0.00060,0.00236,0.0
2,2015-12-11,02:00:00,2,345,4,Friday,2015-12-11T02:00:00.000000000Z,829,1.09327,1.09442,...,0.00166,0.00068,-0.00098,1,-0.00047,0.00115,0.00098,0.00083,0.00060,0.0
3,2015-12-11,06:00:00,6,345,4,Friday,2015-12-11T06:00:00.000000000Z,5337,1.09370,1.09703,...,0.00351,0.00268,-0.00083,1,-0.00065,0.00333,0.00083,-0.00047,0.00083,0.0
4,2015-12-11,10:00:00,10,345,4,Friday,2015-12-11T10:00:00.000000000Z,10131,1.09437,1.09781,...,0.00429,0.00253,-0.00176,1,-0.00091,0.00344,0.00176,-0.00065,-0.00047,0.0


In [26]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5,Trend
9646,2022-02-23,22:00:00,22,54,2,Wednesday,2022-02-23T22:00:00.000000000Z,15853,1.13080,1.13111,...,0.00359,0.00179,-0.00180,0,0.00148,0.00031,0.00180,0.00076,0.00390,-55.0
9647,2022-02-24,02:00:00,2,55,3,Thursday,2022-02-24T02:00:00.000000000Z,62258,1.12932,1.12940,...,0.00854,0.00560,-0.00294,0,0.00552,0.00008,0.00294,0.00148,0.00076,-91.0
9648,2022-02-24,06:00:00,6,55,3,Thursday,2022-02-24T06:00:00.000000000Z,78433,1.12380,1.12624,...,0.00534,0.00510,-0.00024,0,0.00266,0.00244,0.00024,0.00552,0.00148,-94.0
9649,2022-02-24,10:00:00,10,55,3,Thursday,2022-02-24T10:00:00.000000000Z,86397,1.12114,1.12124,...,0.00596,0.00504,-0.00092,0,0.00494,0.00010,0.00092,0.00266,0.00552,-93.0
9650,2022-02-24,14:00:00,14,55,3,Thursday,2022-02-24T14:00:00.000000000Z,35599,1.11619,1.11716,...,0.00394,0.00383,-0.00011,0,0.00286,0.00097,0.00011,0.00494,0.00266,-98.0


In [27]:
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)